In [1]:
from path import Path
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

In [3]:
path = Path('../input')

### Datos de entrenamiento

In [4]:
train_df      = pd.read_csv(path/'train.csv')
train_weather = pd.read_csv(path/"weather_train.csv")

### Datos de test

In [5]:
test_df       = pd.read_csv(path/'test.csv')
test_weather  = pd.read_csv(path/"weather_test.csv")

### Edificios

In [6]:
building_df   = pd.read_csv(path/'building_metadata.csv')

### Opcional: Reducir memoria

In [7]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

train_df = reduce_mem_usage(train_df)
test_df  = reduce_mem_usage(test_df)

train_weather = reduce_mem_usage(train_weather)
test_weather  = reduce_mem_usage(test_weather)

building_df = reduce_mem_usage(building_df)

Mem. usage decreased to 289.19 Mb (53.1% reduction)
Mem. usage decreased to 596.49 Mb (53.1% reduction)
Mem. usage decreased to  3.07 Mb (68.1% reduction)
Mem. usage decreased to  6.08 Mb (68.1% reduction)
Mem. usage decreased to  0.03 Mb (60.3% reduction)


### Metemos las columnas de meta-datos de edificios y climatología

In [8]:
train_df = train_df.merge(building_df, on='building_id', how='left')
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN


In [9]:
train_df = train_df.merge(train_weather, on=['site_id', 'timestamp'], how='left')
train_df.head()

,building_id,meter,timestamp,meter_reading,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,2016-01-01 00:00:00,0.0,0,Education,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
1,1,0,2016-01-01 00:00:00,0.0,0,Education,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
2,2,0,2016-01-01 00:00:00,0.0,0,Education,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
3,3,0,2016-01-01 00:00:00,0.0,0,Education,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
4,4,0,2016-01-01 00:00:00,0.0,0,Education,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0


### Qué atributos vamos a usar

In [14]:
[(c,train_df[c].dtype) for c in train_df.columns]

[('building_id', dtype('int16')),
 ('meter', dtype('int8')),
 ('timestamp', dtype('O')),
 ('meter_reading', dtype('float32')),
 ('site_id', dtype('int8')),
 ('primary_use', dtype('O')),
 ('square_feet', dtype('int32')),
 ('year_built', dtype('float16')),
 ('floor_count', dtype('float16')),
 ('air_temperature', dtype('float16')),
 ('cloud_coverage', dtype('float16')),
 ('dew_temperature', dtype('float16')),
 ('precip_depth_1_hr', dtype('float16')),
 ('sea_level_pressure', dtype('float16')),
 ('wind_direction', dtype('float16')),
 ('wind_speed', dtype('float16'))]

In [16]:
train_df["timestamp"] = pd.to_datetime(train_df["timestamp"])
train_df["hour"]      = train_df["timestamp"].dt.hour
train_df["month"]     = train_df["timestamp"].dt.month
train_df["weekday"]   = train_df["timestamp"].dt.weekday
#x['year_built'] = x['year_built']-1900
#x['square_feet'] = np.log(x['square_feet'])

In [18]:
cat_features  = ['building_id','meter', 'site_id','primary_use', 'hour', 'weekday', 'month' ]

cont_features = ['square_feet', 'year_built', 'floor_count', 'air_temperature', 
                 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 
                 'sea_level_pressure', 'wind_direction', 'wind_speed']
                 

In [30]:
for c in cat_features:
    train_df[c] = train_df[c].astype('category')

### Definimos la x (entrada) y la y (salida o valor a predecir)

In [48]:
x = train_df[cat_features + cont_features]
y = np.log1p(train_df[['meter_reading']].values.squeeze(-1))

In [50]:
x.head()

,building_id,meter,site_id,primary_use,hour,weekday,month,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,Education,0,4,1,7432,2008.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
1,1,0,0,Education,0,4,1,2720,2004.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
2,2,0,0,Education,0,4,1,5376,1991.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
3,3,0,0,Education,0,4,1,23685,2002.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0
4,4,0,0,Education,0,4,1,116607,1975.0,NaN,25.0,6.0,20.0,NaN,1019.5,0.0,0.0


In [51]:
x.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20216100 entries, 0 to 20216099
Data columns (total 17 columns):
building_id           category
meter                 category
site_id               category
primary_use           category
hour                  category
weekday               category
month                 category
square_feet           int32
year_built            float16
floor_count           float16
air_temperature       float16
cloud_coverage        float16
dew_temperature       float16
precip_depth_1_hr     float16
sea_level_pressure    float16
wind_direction        float16
wind_speed            float16
dtypes: category(7), float16(9), int32(1)
memory usage: 732.7 MB


### Segmentación estratificada

In [52]:
import lightgbm as lgb
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error

In [53]:
params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'rmse'},
            'subsample': 0.25,
            'subsample_freq': 1,
            'learning_rate': 0.4,
            'num_leaves': 20,
            'feature_fraction': 0.9,
            'lambda_l1': 1,  
            'lambda_l2': 1
            }

In [55]:
folds = 4
seed = 42

kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seed)

models = []
for fold,(train_idx, val_idx) in enumerate(tqdm(kf.split(train_df, train_df['building_id']),
                                               total=folds)):
    x_train,x_val = x.iloc[train_idx], x.iloc[val_idx]
    y_train,y_val =      y[train_idx],      y[val_idx]

    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_val   = lgb.Dataset(x_val,   y_val)
    
    gbm = lgb.train(
        params, lgb_train, num_boost_round=1000, valid_sets=(lgb_train, lgb_val),
        early_stopping_rounds=100, verbose_eval = 100, )
    
    models.append(gbm)
    log_preds_val = gbm.predict(x_val)
    print(f'Fold: {fold} VAL RMSLE: {np.sqrt(mean_squared_error( y_val, log_preds_val))}')


  0%|          | 0/4 [00:00<?, ?it/s]/home/antor/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 0.912991	valid_1's rmse: 0.913567
[200]	training's rmse: 0.846162	valid_1's rmse: 0.84737
[300]	training's rmse: 0.812019	valid_1's rmse: 0.813706
[400]	training's rmse: 0.790038	valid_1's rmse: 0.792282
[500]	training's rmse: 0.772888	valid_1's rmse: 0.775718
[600]	training's rmse: 0.759156	valid_1's rmse: 0.76264
[700]	training's rmse: 0.747351	valid_1's rmse: 0.751383
[800]	training's rmse: 0.738192	valid_1's rmse: 0.742879
[900]	training's rmse: 0.731589	valid_1's rmse: 0.736785
[1000]	training's rmse: 0.72518	valid_1's rmse: 0.730935
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 0.72518	valid_1's rmse: 0.730935



 25%|██▌       | 1/4 [14:36<43:50, 876.88s/it]

Fold: 0 VAL RMSLE: 0.7309354548907259


/home/antor/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


...

Fold: 2 VAL RMSLE: 0.7356400792272263


/home/antor/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 0.914436	valid_1's rmse: 0.915632
[200]	training's rmse: 0.847195	valid_1's rmse: 0.849262
[300]	training's rmse: 0.81483	valid_1's rmse: 0.817593
[400]	training's rmse: 0.791967	valid_1's rmse: 0.795472
[500]	training's rmse: 0.777644	valid_1's rmse: 0.781638
[600]	training's rmse: 0.762641	valid_1's rmse: 0.767215
[700]	training's rmse: 0.752075	valid_1's rmse: 0.757226
[800]	training's rmse: 0.741593	valid_1's rmse: 0.747388
[900]	training's rmse: 0.733747	valid_1's rmse: 0.740233
[1000]	training's rmse: 0.727108	valid_1's rmse: 0.734367
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 0.727108	valid_1's rmse: 0.734367



100%|██████████| 4/4 [35:14<00:00, 572.00s/it]

Fold: 3 VAL RMSLE: 0.7343666019870063


In [56]:
models

### Metemos las columnas de meta-datos de edificios y climatología (en test)

In [57]:
test_df = test_df.merge(building_df, on='building_id', how='left')
test_df = test_df.merge(test_weather, on=['site_id', 'timestamp'], how='left')
test_df.head()

,row_id,building_id,meter,timestamp,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,2017-01-01 00:00:00,0,Education,7432,2008.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
1,1,1,0,2017-01-01 00:00:00,0,Education,2720,2004.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
2,2,2,0,2017-01-01 00:00:00,0,Education,5376,1991.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
3,3,3,0,2017-01-01 00:00:00,0,Education,23685,2002.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
4,4,4,0,2017-01-01 00:00:00,0,Education,116607,1975.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609


In [59]:
test_df["timestamp"] = pd.to_datetime(test_df["timestamp"])
test_df["hour"]      = test_df["timestamp"].dt.hour
test_df["month"]     = test_df["timestamp"].dt.month
test_df["weekday"]   = test_df["timestamp"].dt.weekday

In [61]:
for c in cat_features:
    test_df[c] = test_df[c].astype('category')

In [62]:
test = test_df[cat_features + cont_features]
test

,building_id,meter,site_id,primary_use,hour,weekday,month,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,Education,0,6,1,7432,2008.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
1,1,0,0,Education,0,6,1,2720,2004.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
2,2,0,0,Education,0,6,1,5376,1991.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
3,3,0,0,Education,0,6,1,23685,2002.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
4,4,0,0,Education,0,6,1,116607,1975.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
5,5,0,0,Education,0,6,1,8000,2000.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
6,6,0,0,Lodging/residential,0,6,1,27926,1981.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
7,7,0,0,Education,0,6,1,121074,1989.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
8,7,1,0,Education,0,6,1,121074,1989.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609
9,8,0,0,Education,0,6,1,60809,2003.0,NaN,17.796875,4.0,11.703125,NaN,1021.5,100.0,3.599609


In [70]:
[(c,train_df[c].dtype) for c in train_df.columns]

[('building_id',
  CategoricalDtype(categories=[   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
                    ...
                    1439, 1440, 1441, 1442, 1443, 1444, 1445, 1446, 1447, 1448],
                   ordered=False)),
 ('meter', CategoricalDtype(categories=[0, 1, 2, 3], ordered=False)),
 ('timestamp', dtype('<M8[ns]')),
 ('meter_reading', dtype('float32')),
 ('site_id',
  CategoricalDtype(categories=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], ordered=False)),
 ('primary_use',
  CategoricalDtype(categories=['Education', 'Entertainment/public assembly',
                    'Food sales and service', 'Healthcare',
                    'Lodging/residential', 'Manufacturing/industrial', 'Office',
                    'Other', 'Parking', 'Public services', 'Religious worship',
                    'Retail', 'Services', 'Technology/science', 'Utility',
                    'Warehouse/storage'],
                   ordered=False)),
 ('square_feet', dtype('i

In [69]:
[(c,test_df[c].dtype) for c in test_df.columns]

[('row_id', dtype('int32')),
 ('building_id',
  CategoricalDtype(categories=[   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
                    ...
                    1439, 1440, 1441, 1442, 1443, 1444, 1445, 1446, 1447, 1448],
                   ordered=False)),
 ('meter', CategoricalDtype(categories=[0, 1, 2, 3], ordered=False)),
 ('timestamp', dtype('<M8[ns]')),
 ('site_id',
  CategoricalDtype(categories=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], ordered=False)),
 ('primary_use',
  CategoricalDtype(categories=['Education', 'Entertainment/public assembly',
                    'Food sales and service', 'Healthcare',
                    'Lodging/residential', 'Manufacturing/industrial', 'Office',
                    'Other', 'Parking', 'Public services', 'Religious worship',
                    'Retail', 'Services', 'Technology/science', 'Utility',
                    'Warehouse/storage'],
                   ordered=False)),
 ('square_feet', dtype('int32')),


### Predicciones

In [71]:
np.expm1(models[0].predict(x))

array([ 2.09741860e-01,  1.59974784e-01, -7.97420626e-03, ...,
        9.84137575e-02,  1.92959614e+02,  2.74691087e+00])

In [72]:
y

array([0.       , 0.       , 0.       , ..., 0.       , 5.078761 ,
       1.3480731], dtype=float32)

### Mezclamos (ensemble) los modelos (media)

In [64]:
models_preds = np.array([np.expm1(model.predict(test)) for model in models])
models_preds, models_preds.shape

(array([[ 7.39363358e-02,  1.83088473e-01, -1.56045724e-02, ...,
          5.46663590e+00,  2.17699706e+02,  3.43089005e+00],
        [ 2.22346171e-01,  2.78384975e-01, -1.38702378e-01, ...,
          2.36582781e+00,  1.91492370e+02,  3.08873313e+00],
        [ 7.64803286e-01,  2.42666452e-01, -2.53674414e-01, ...,
          4.15522772e+00,  1.64732098e+02,  3.91526774e+00],
        [ 3.79408296e-01,  2.09328890e-02, -1.52732024e-01, ...,
          2.53251492e+00,  1.45203243e+02,  3.44369364e+00]]), (4, 41697600))

In [65]:
preds = np.mean(models_preds,axis=0)
preds

array([ 3.60123522e-01,  1.81268197e-01, -1.40178347e-01, ...,
        3.63005159e+00,  1.79781854e+02,  3.46964614e+00])

In [83]:
preds = np.clip(preds,0,None)
preds

array([  0.36012352,   0.1812682 ,   0.        , ...,   3.63005159,
       179.78185424,   3.46964614])

### Preparemos el envío

In [84]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
sample_submission

,row_id,meter_reading
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
5,5,0
6,6,0
7,7,0
8,8,0
9,9,0


In [86]:
sample_submission['meter_reading'] = preds
sample_submission

,row_id,meter_reading
0,0,0.360124
1,1,0.181268
2,2,0.000000
3,3,0.258549
4,4,0.899362
5,5,0.000000
6,6,0.117204
7,7,0.053983
8,8,286.971131
9,9,0.000000


### A disco en un CSV de toda la vida

In [87]:
csv_fname = '4-lgbm-kfold-estratificado-con-nans-y-cats.csv.zip'
sample_submission.to_csv(csv_fname, compression='zip', index=False)

### Opcional: enviar a Kaggle (OJO! que hay limite diario)

In [88]:
comp = 'ashrae-energy-prediction'
!kaggle competitions submit -c {comp} -f {csv_fname} -m 'lgbm con 4 folds (NaNs+cats)'

100%|████████████████████████████████████████| 467M/467M [00:10<00:00, 45.0MB/s]
Successfully submitted to ASHRAE - Great Energy Predictor III

In [89]:
# LB 1.37

### ¿Por qué el LB es 1.37 y el CV ~0.73?